In [ ]:
import json
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from tqdm import tqdm
from scipy import stats
from sklearn import metrics
from IPython.display import display
from ki67.interfaces.cells import Cells

In [ ]:
sns.set_theme(style='whitegrid')

### Metrics calculation

In [ ]:
INVALID = 0.2

In [ ]:
with open('./data/experiments/config.json') as fp:
    config = json.load(fp)

slides = {
    'amy': config['shards']['amy'] + config['testing'],
    'ben': config['shards']['ben'] + config['testing'],
    'charlie': config['shards']['charlie'] + config['testing'],
    'ensemble': config['testing'],
    'identity': config['shards']['amy'] + config['shards']['ben'] + config['shards']['charlie'] + config['testing'],
    'pathonet': config['testing'],
}

In [ ]:
def process_single(slide: str, model: str, size: int, biased: bool, gt_ki: float) -> pd.Series:
    if model == 'identity':
        cells = Cells.parse(slide, np.load(f'data/results/{slide}/cells-identity.npz'))
    elif model == 'pathonet':
        pathonet = pd.read_csv(f'data/experiments/pathonet.csv')
    else:
        prefix = 'biased-cells' if biased else 'cells'
        filename = f'{prefix}-f{size}-{model}.npz'
        cells = Cells.parse(slide, np.load(f'data/results/{slide}/{filename}'))

    if model != 'pathonet':
        positive_area = np.sum(cells.data[cells.CellType.POSITIVE].mask)
        cumulative_area = np.sum(cells.data[cells.CellType.ALL].mask)
        ki = positive_area / cumulative_area if cumulative_area > 0 else np.NaN
    else:
        ki = pathonet[pathonet.slide == slide].iloc[0]['ki']

    return pd.Series(dict(
        slide=slide,
        model=model,
        fragment=size,
        biased=biased,
        gt=gt_ki,
        ki=ki,
        error=gt_ki - ki,
        delta=np.abs(gt_ki - ki),
    ))

def process_slide(slide: str, model: str) -> pd.DataFrame:
    markers = pd.read_parquet(f'data/results/{slide}/markers.gz.parquet')
    gt_ki = len(markers[markers.type == 1]) / len(markers)

    return (
        pd.DataFrame([
            process_single(slide, model, size, biased, gt_ki)
            for size in [48, 96, 192]
            for biased in [True, False]
        ]) if model != 'identity' and model != 'pathonet'
        else pd.DataFrame([process_single(slide, model, np.nan, np.nan, gt_ki)])
    )

def calculate_metrics(df: pd.DataFrame, confidence_level = 0.95) -> pd.Series:
    degrees_freedom = len(df) - 1
    sample_mean = np.mean(df.delta)
    sample_standard_error = stats.sem(df.delta)
    confidence_interval = stats.t.interval(confidence_level, degrees_freedom, sample_mean, sample_standard_error)

    return pd.Series(dict(
        mae=df.delta.mean(),
        ci=f'{max(confidence_interval[0], 0.0):.3f} to {confidence_interval[1]:.3f}',
        rmse=metrics.mean_squared_error(df['gt'], df['ki'], squared=False),
        invalid=len(df[(df.delta > INVALID) | (df.delta.isna())])
    ))


In [ ]:
data = pd.concat([
    process_slide(slide, model)
    for slide, model in tqdm([
        (slide, model)
        for model in slides.keys()
        for slide in slides[model]
    ])],
    ignore_index=True,
)
data.to_csv('data/experiments/ki67.csv', index=False)

In [ ]:
data = pd.read_csv('data/experiments/ki67.csv')
data

In [ ]:
identity = data[data.model == 'identity']
pathonet = data[data.model == 'pathonet']
single = data[data.model.isin(['amy', 'ben', 'charlie'])].groupby(['slide', 'fragment', 'biased']).mean().reset_index()
ensemble = data[data.model == 'ensemble']

### Results

In [ ]:
display(
    pd.concat([
        calculate_metrics(identity),
        calculate_metrics(identity[identity.slide.isin(config['testing'])]),
    ], axis=1, keys=['all', 'testing'])
)

In [ ]:
display(
    pd.DataFrame([
        calculate_metrics(pathonet[pathonet.slide.isin(config['testing'])]),
    ])
)

In [ ]:
pd.concat([
    single.groupby('fragment').apply(
        lambda df: df.groupby('biased').apply(
            lambda dff: calculate_metrics(dff)
        ).T
    ),
    single[single.slide.isin(config['testing'])].groupby('fragment').apply(
        lambda df: df.groupby('biased').apply(
            lambda dff: calculate_metrics(dff)
        ).T
    ),
], axis=1, keys=['all', 'testing'])

In [ ]:
ensemble.groupby('fragment').apply(
    lambda df: df.groupby('biased').apply(
        lambda dff: calculate_metrics(dff)
    ).T
)

### Plots

In [ ]:
df = pd.concat([
    single[single.slide.isin(config['testing'])].assign(Model=lambda dff: dff.apply(
        lambda row: 'Single' + (' Biased' if row['biased'] else ''),
        axis=1,
    )),
    ensemble.assign(Model=lambda dff: dff.apply(
        lambda row: 'Ensemble' + (' Biased' if row['biased'] else ''),
        axis=1,
    )),
])

fig, ax = plt.subplots(figsize=(6, 4))
g = sns.boxplot(
    x='fragment',
    y='delta',
    hue='Model',
    hue_order=['Single', 'Ensemble', 'Single Biased', 'Ensemble Biased'],
    data=df,
    showfliers=False,
    ax=ax,
)
g.set_xlabel('Window size')
g.set_ylabel('MAE')
g.set_xticklabels(['48 px', '96 px', '192 px'])
None

In [ ]:
df = pd.concat([
    single[single.slide.isin(config['testing'])].assign(Model='Single', Biased=lambda dff: dff.biased.astype(bool)),
    ensemble.assign(Model='Ensemble', Biased=lambda dff: dff.biased.astype(bool)),
])

fig, ax = plt.subplots(figsize=(5, 4))
g = sns.violinplot(
    x='Model',
    y='delta',
    hue='Biased',
    data=df,
    split=False,
    cut=0,
    scale_hue=True,
    ax=ax,
)
g.set_ylabel('MAE')
None

In [ ]:
df = pd.concat([
    single[(single.fragment == 96) & (single.biased == True)]
        .assign(Model='Single Biased (96px window)'),
    ensemble[(ensemble.fragment == 96) & (ensemble.biased == False)]
        .assign(Model='Ensemble Unbiased (96px window)'),
])

g = sns.lmplot(
    data=df,
    x='gt',
    y='ki',
    hue='Model',
    col='Model',
    height=4,
    truncate=False,
)
g.set_axis_labels('Ground Truth Ki67 PI', 'Estimated Ki67 PI')

for ax in g.axes[0]:
    ax.plot([0, 1], [0, 1], 'g-')

In [ ]:
df = pd.concat([
    identity.assign(Model='Base'),
    pathonet.assign(Model='PathoNet'),
    ensemble[(ensemble.fragment == 96) & (ensemble.biased == False)].assign(Model='Our'),
])

fig, ax = plt.subplots(figsize=(2.5, 4))
g = sns.boxplot(
    x='Model',
    y='delta',
    data=df,
    showfliers=False,
    ax=ax,
)
g.set_xlabel('Solution')
g.set_ylabel('MAE')
None